In [1]:
import sys
import pandas as pd
import os

repo_dir = "/Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/"

data_dir = os.path.join(repo_dir, 'src/data/mcpherson_ovarian_2016')

In [2]:
# (1) First, read in WGS reads data from McPherson and reformat into 
# file structure needed for MACHINA's and PyClone's clustering
# TODO: how to handle "status" column? -> shows somatic, wildtype, germline?

# Taken directly from supplementary table 10 of McPherson et.al.
reads_df = pd.read_csv(os.path.join(data_dir, 'supplement_table_10.csv'))
patient_ids = reads_df['patient_id'].unique()
print("patient IDs:", patient_ids)
reads_df['gene_name'] = reads_df['gene_name'].astype(str)
def label_snv(row):
    label = []
    if row['gene_name'] != "nan":
        label.append(row['gene_name'])
    label += [str(row['chrom']), str(row['coord'])]
    return (":").join(label)
reads_df['character_label'] = reads_df.apply(lambda row: label_snv(row), axis=1)
reads_df

patient IDs: [ 1  2  3  4  7  9 10]


,sample_id,patient_id,malignant,primer_id,chrom,coord,ref,alt,ref_counts,alt_counts,depth,alt_freq,background_average_alt_freq,ref_p_value,alt_p_value,status,gene_name,snpeff_impact,character_label
0,normal_blood,1,no,1_b_amplicrazy,1,17090971,G,A,641,0,641,0.000000,0.002034,0.0,1.000000e+00,normal_sample,nan,NaN,1:17090971
1,omentum_site_1,1,yes,1_b_amplicrazy,1,17090971,G,A,449,0,449,0.000000,0.001597,0.0,1.000000e+00,wildtype,nan,NaN,1:17090971
2,right_ovary_site_1,1,yes,1_b_amplicrazy,1,17090971,G,A,395,64,459,0.139434,0.002375,0.0,7.795800e-90,somatic,nan,NaN,1:17090971
3,right_ovary_site_2,1,yes,1_b_amplicrazy,1,17090971,G,A,415,1,416,0.002404,0.002754,0.0,6.825264e-01,wildtype,nan,NaN,1:17090971
4,right_ovary_site_3,1,yes,1_b_amplicrazy,1,17090971,G,A,609,4,613,0.006525,0.002145,0.0,4.440635e-02,wildtype,nan,NaN,1:17090971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15293,normal_blood,10,no,10_b_amplicrazy,X,109694487,A,G,3669,10,3679,0.002718,0.002239,0.0,3.130859e-01,normal_sample,RGAG1,LOW,RGAG1:X:109694487
15294,right_ovary_site_1,10,yes,10_b_amplicrazy,X,109694487,A,G,2037,295,2332,0.126501,0.002306,0.0,0.000000e+00,somatic,RGAG1,LOW,RGAG1:X:109694487
15295,right_ovary_site_2,10,yes,10_b_amplicrazy,X,109694487,A,G,1369,4,1373,0.002913,0.002442,0.0,4.314746e-01,wildtype,RGAG1,LOW,RGAG1:X:109694487
15296,right_ovary_site_3,10,yes,10_b_amplicrazy,X,109694487,A,G,2675,4,2679,0.001493,0.002189,0.0,8.364492e-01,wildtype,RGAG1,LOW,RGAG1:X:109694487


In [3]:
# Load the table that contains which "high-quality DNA extractions (discovery samples)"
# were used in the paper's analysis
discovery_samples_df = pd.read_csv(os.path.join(data_dir, 'supplement_table_2.csv'))
discovery_samples_df

,patient_id,sample_id,paper_id,total_reads,aligned_reads,coverage,mutation_seq_snvs,strelka_snvs,all_snvs,high_quality_snvs,validated_snvs,ploidy,tumour_cell_proportion,subclone_frequency
0,1,normal_blood,Nml,1288936606,1045839192,34.477126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,omentum_site_1,Om1,987945944,884565254,28.788908,5715.0,6665.0,7281.0,3550.0,120.0,3.606594,0.463669,0.318225
2,1,right_ovary_site_1,ROv1,1261617726,1076429948,34.996814,6276.0,7902.0,8514.0,3986.0,132.0,1.880553,0.826552,0.251911
3,1,right_ovary_site_2,ROv2,1389016408,1194767689,38.785042,6004.0,7256.0,7912.0,3757.0,111.0,1.847366,0.695751,0.268036
4,1,right_ovary_site_3,ROv3,1205431286,1056428422,33.312377,5803.0,6756.0,7410.0,3612.0,118.0,1.854135,0.678331,0.265984
5,1,right_ovary_site_4,ROv4,1480564778,1300349864,36.794602,6501.0,7920.0,8664.0,4017.0,112.0,1.857220,0.702755,0.240692
6,1,small_bowel_site_1,SBwl,1242912402,1096324079,35.650173,7100.0,8323.0,9137.0,4357.0,127.0,3.611431,0.448979,0.270676
7,2,normal_blood,Nml,1395804938,1194671812,37.449153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2,omentum_site_1,Om1,1058316688,936624472,30.631961,1860.0,2705.0,2849.0,1228.0,106.0,3.016544,0.196412,0.347316
9,2,omentum_site_2,Om2,1191965188,1070042684,34.932824,3046.0,4036.0,4242.0,1980.0,107.0,3.014453,0.239950,0.348032


In [4]:
discovery_samples_df[discovery_samples_df['patient_id']==1]['sample_id']

0          normal_blood
1        omentum_site_1
2    right_ovary_site_1
3    right_ovary_site_2
4    right_ovary_site_3
5    right_ovary_site_4
6    small_bowel_site_1
Name: sample_id, dtype: object

In [5]:
# (2a) prep data for MACHINA clustering
cols = ['sample_id', 'ref_counts', 'alt_counts', 'character_label']

# put df in format used in MACHINA, with columns:
#'#sample_index', 'sample_label','anatomical_site_index', 'anatomical_site_label', 'character_index', 'character_label', 'ref', 'var'
for patient_id in patient_ids:
    subset = reads_df[reads_df['patient_id'] == patient_id][cols]
    # only keep the high quality discovery samples used in the paper
    discovery_samples = discovery_samples_df[discovery_samples_df['patient_id']==patient_id]['sample_id']
    subset = subset[subset['sample_id'].isin(discovery_samples)]
    # remove normal samples
    subset = subset[subset['sample_id'] != 'normal_blood'] 
    subset = subset.rename(columns={"ref_counts": "ref", "alt_counts": "var", "sample_id": "sample_label"})
    char_labels = list(subset['character_label'].unique())
    sites = list(subset['sample_label'].unique())
    subset['character_index'] = subset.apply(lambda row: char_labels.index(row['character_label']), axis=1)
    subset['#sample_index'] = subset.apply(lambda row: sites.index(row['sample_label']), axis=1)
    subset['anatomical_site_index'] = subset['#sample_index']
    subset['anatomical_site_label'] = subset['sample_label']
    subset = subset[['#sample_index', 'sample_label','anatomical_site_index', 'anatomical_site_label', 'character_index', 'character_label', 'ref', 'var']]
    subset.to_csv(os.path.join(data_dir,  f"reads_patient{patient_id}.csv"), index=False)


In [6]:
# (2b) prep data for PyClone clustering
# get CNA info from supplement of McPherson et. al.
cna_df = pd.read_csv(os.path.join(data_dir, 'supplement_table_7.csv'))
print(cna_df)

def subset_cna(row):
    chrom = row['chrom']
    coord = row['coord']
    patient_id = row['patient_id']
    sample_id = row['sample_id']
    df = cna_df[(cna_df['patient_id'] == patient_id) & (cna_df['sample_id'] == sample_id) & (cna_df['chrom'] == chrom) & (cna_df['start'] <= coord) & (cna_df['end'] >= coord)]
    return df

def major_cn(row):
    coord_df = subset_cna(row)
    if len(coord_df) != 1:
        #print("Not found in CNA:\n", row['chrom'], row['coord'])
        return 0
    return int(coord_df['major'])

def minor_cn(row):
    coord_df = subset_cna(row)
    if len(coord_df) != 1:
        return 0
    return int(coord_df['minor'])


# Add CNAs to reads_df
reads_df['major_cn'] =  reads_df.apply(lambda row: major_cn(row), axis=1)
reads_df['minor_cn'] =  reads_df.apply(lambda row: minor_cn(row), axis=1)
reads_df

        patient_id           sample_id chrom      start        end  major  \
0                1  small_bowel_site_1     1          1      71283     16   
1                1  small_bowel_site_1     1      73560      94700      7   
2                1  small_bowel_site_1     1     101640     110479     13   
3                1  small_bowel_site_1     1     110479     141357     22   
4                1  small_bowel_site_1     1     144162     247926     24   
...            ...                 ...   ...        ...        ...    ...   
135154          10  right_ovary_site_4     X  152951113  152955404      0   
135155          10  right_ovary_site_4     X  152955404  154151513      2   
135156          10  right_ovary_site_4     X  154151513  154887231      2   
135157          10  right_ovary_site_4     X  154887231  154915957      2   
135158          10  right_ovary_site_4     X  154915957  155255695      2   

        minor  major_sub  minor_sub  subclonal  
0           7          0  

,sample_id,patient_id,malignant,primer_id,chrom,coord,ref,alt,ref_counts,alt_counts,...,alt_freq,background_average_alt_freq,ref_p_value,alt_p_value,status,gene_name,snpeff_impact,character_label,major_cn,minor_cn
0,normal_blood,1,no,1_b_amplicrazy,1,17090971,G,A,641,0,...,0.000000,0.002034,0.0,1.000000e+00,normal_sample,nan,NaN,1:17090971,0,0
1,omentum_site_1,1,yes,1_b_amplicrazy,1,17090971,G,A,449,0,...,0.000000,0.001597,0.0,1.000000e+00,wildtype,nan,NaN,1:17090971,13,5
2,right_ovary_site_1,1,yes,1_b_amplicrazy,1,17090971,G,A,395,64,...,0.139434,0.002375,0.0,7.795800e-90,somatic,nan,NaN,1:17090971,4,2
3,right_ovary_site_2,1,yes,1_b_amplicrazy,1,17090971,G,A,415,1,...,0.002404,0.002754,0.0,6.825264e-01,wildtype,nan,NaN,1:17090971,6,2
4,right_ovary_site_3,1,yes,1_b_amplicrazy,1,17090971,G,A,609,4,...,0.006525,0.002145,0.0,4.440635e-02,wildtype,nan,NaN,1:17090971,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15293,normal_blood,10,no,10_b_amplicrazy,X,109694487,A,G,3669,10,...,0.002718,0.002239,0.0,3.130859e-01,normal_sample,RGAG1,LOW,RGAG1:X:109694487,0,0
15294,right_ovary_site_1,10,yes,10_b_amplicrazy,X,109694487,A,G,2037,295,...,0.126501,0.002306,0.0,0.000000e+00,somatic,RGAG1,LOW,RGAG1:X:109694487,1,0
15295,right_ovary_site_2,10,yes,10_b_amplicrazy,X,109694487,A,G,1369,4,...,0.002913,0.002442,0.0,4.314746e-01,wildtype,RGAG1,LOW,RGAG1:X:109694487,1,0
15296,right_ovary_site_3,10,yes,10_b_amplicrazy,X,109694487,A,G,2675,4,...,0.001493,0.002189,0.0,8.364492e-01,wildtype,RGAG1,LOW,RGAG1:X:109694487,1,0


In [7]:
# do some pre-processing to remove any mutations where CNA info is not available or
# the major allele CN is 0 (if the malignant cells have no copies of the region 
# overlapping the mutation, the mutation cannot exist.) or minor allele CN > major allele CN
reads_df = reads_df[reads_df['major_cn'] != 0]
reads_df = reads_df[(reads_df['minor_cn'] <= reads_df['major_cn'])]
reads_df['normal_cn'] = 2 # copy number of the locus in normal cells is 2 (no male chromosomes here)

pyclone_dir = os.path.join(data_dir, "pyclone_preprocessing")

cols = ['sample_id', 'ref_counts', 'alt_counts', 'character_label', "chrom", "coord", "major_cn", "minor_cn", "normal_cn"]
# put df in format used in PyClone, with columns:
#'mutation_id', 'ref_counts','var_counts', 'normal_cn', 'minor_cn', 'major_cn'
for patient_id in patient_ids:
    patient_dir = os.path.join(pyclone_dir, f"reads_patient_{patient_id}")
    if not os.path.exists(patient_dir): 
        os.makedirs(patient_dir)
    patient_subset = reads_df[reads_df['patient_id'] == patient_id][cols]
    # only keep the high quality discovery samples used in the paper
    discovery_samples = discovery_samples_df[discovery_samples_df['patient_id']==patient_id]['sample_id']
    patient_subset = patient_subset[patient_subset['sample_id'].isin(discovery_samples)]
    # remove normal samples
    patient_subset = patient_subset[patient_subset['sample_id'] != 'normal_blood'] 
    sample_ids = list(patient_subset['sample_id'].unique())
    patient_subset = patient_subset.rename(columns={"ref_counts": "ref_counts", "alt_counts": "var_counts", "character_label": "mutation_id"})
    
    for sample_id in sample_ids:
        sample_subset = patient_subset[patient_subset['sample_id'] == sample_id]
        sample_subset = sample_subset[['mutation_id', 'ref_counts', 'var_counts', 'normal_cn', 'minor_cn', 'major_cn']]
        sample_subset.to_csv(os.path.join(patient_dir,  f"reads_patient{patient_id}_{sample_id}.csv"))    


In [8]:
# (3) Setup PyClone commands to run for each patient

for patient_id in patient_ids:
    cmd = ["PyClone", "run_analysis_pipeline", "--in_files"]
    patient_dir = os.path.join(pyclone_dir, f"reads_patient_{patient_id}")
    discovery_samples = list(discovery_samples_df[discovery_samples_df['patient_id']==patient_id]['sample_id'])
    discovery_samples.remove("normal_blood")
    # Add tsvs for each sample
    for sample_id in discovery_samples:
        cmd.append(os.path.join(patient_dir,  f"reads_patient{patient_id}_{sample_id}.tsv"))
    cmd += ["--working_dir", patient_dir, "--tumour_contents"]
    # Add tumour cell proportions for each sample
    for sample_id in discovery_samples:
        tumour_prop_subset = discovery_samples_df[(discovery_samples_df['patient_id']==patient_id) & (discovery_samples_df['sample_id']==sample_id)]
        tumour_prop = tumour_prop_subset['tumour_cell_proportion']                                                                                            
        cmd.append(str(tumour_prop.values[0]))
    # Add sample names
    cmd.append("--samples")
    for sample_id in discovery_samples:
        cmd.append(sample_id)
    print(" ".join(cmd))
    

PyClone run_analysis_pipeline --in_files /Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/src/data/mcpherson_ovarian_2016/pyclone_preprocessing/reads_patient_1/reads_patient1_omentum_site_1.tsv /Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/src/data/mcpherson_ovarian_2016/pyclone_preprocessing/reads_patient_1/reads_patient1_right_ovary_site_1.tsv /Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/src/data/mcpherson_ovarian_2016/pyclone_preprocessing/reads_patient_1/reads_patient1_right_ovary_site_2.tsv /Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/src/data/mcpherson_ovarian_2016/pyclone_preprocessing/reads_patient_1/reads_patient1_right_ovary_site_3.tsv /Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/src/data/mcpherson_ovarian_2016/pyclone_preprocessing/reads_patient_1/reads_patient1_right_ovary_site_4.tsv /Us

In [9]:
# (4a) Write results of PyClone clustering out to file to use as input for SPRUCE (tree inference)
# For each patient, write a file where each line contains the mutations in each cluster

for patient_id in patient_ids:
    patient_dir = os.path.join(pyclone_dir, f"reads_patient_{patient_id}", "tables")
    cluster_df = pd.read_csv(os.path.join(patient_dir, 'loci.tsv'), sep = "\t")
    clusters = cluster_df['cluster_id'].unique()
    clusters.sort()
    assert((clusters[0] == 0) and (clusters[-1] == len(clusters)-1))
    with open(os.path.join(pyclone_dir, f"patient{patient_id}_pyclone_clusters.txt"), 'w') as f:
        for cluster in clusters:
            subset = cluster_df[cluster_df['cluster_id']==cluster]
            mutations = subset['mutation_id'].unique()
            f.write(";".join(mutations))
            f.write("\n")
            

In [3]:
# (4b) 

# TODO: remove
pyclone_dir = os.path.join(data_dir, "pyclone_preprocessing")


repo_dir = "/Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/"
os.chdir(repo_dir)

from src.util import create_conf_intervals_from_reads as conf

# Create tsv for each patient with ref reads, var reads, and f upper bound and f lower bound
# pooled by all SNVs in a cluster
cluster_split = lambda cluster_name: [x for x in cluster_name.split(";")]

for patient_id in patient_ids:
    patient_dir = os.path.join(pyclone_dir, f"reads_patient_{patient_id}")
    conf.write(os.path.join(data_dir, f"reads_patient{patient_id}.csv"),
               os.path.join(pyclone_dir, f"patient{patient_id}_pyclone_clusters.txt"),
               data_dir,
               cluster_split_function=cluster_split)


num variants: 299
anatomical site labels: ['omentum_site_1' 'right_ovary_site_1' 'right_ovary_site_2'
 'right_ovary_site_3' 'right_ovary_site_4' 'small_bowel_site_1']
[0.00133168 0.0013338  0.00133592 ... 0.00344663 0.00344875 0.00345087]
[0.23325382 0.2332645  0.23327519 ... 0.24391938 0.24393007 0.24394075]
[0.00011765 0.00011819 0.00011874 ... 0.00066519 0.00066574 0.00066628]
[0.03454669 0.0345671  0.03458751 ... 0.05491536 0.05493577 0.05495618]
[0.40057801 0.40058832 0.40059862 ... 0.41086317 0.41087348 0.41088379]
[-0.55119819 -0.55009579 -0.5489934  ...  0.5489934   0.55009579
  0.55119819]
[4.42467555e-05 4.48381328e-05 4.54295102e-05 ... 6.34441314e-04
 6.35032692e-04 6.35624069e-04]
[0.1936639  0.19368806 0.19371222 ... 0.21777513 0.21779929 0.21782345]
[4.99088556e-05 5.04516540e-05 5.09944525e-05 ... 5.91621709e-04
 5.92164508e-04 5.92707306e-04]
[2.02731218e-05 2.12811076e-05 2.22890934e-05 ... 1.02624296e-03
 1.02725095e-03 1.02825893e-03]
[0.00030767 0.0003082  0.000308

/Users/divyakoyyalagunta/Desktop/Cornell_Research/Morris_Lab/met_history_prediction/src/util/create_conf_intervals_from_reads.py:50: RuntimeWarning: invalid value encountered in double_scalars
  mode = (n+a-1.)/(N+a+b-2.)


[0.25265083 0.25267273 0.25269464 ... 0.2745138  0.27453571 0.27455762]
[0.00033045 0.00033111 0.00033177 ... 0.00098949 0.00099015 0.00099081]
[0.30591892 0.30594456 0.30597021 ... 0.33151426 0.33153991 0.33156556]
[0.11220583 0.11251027 0.1128147  ... 0.41603145 0.41633589 0.41664032]
[-0.00014084 -0.00014029 -0.00013973 ...  0.00041588  0.00041644
  0.00041699]
[0.31743136 0.31745204 0.31747271 ... 0.33806775 0.33808843 0.3381091 ]
[-0.70429521 -0.70288662 -0.70147803 ...  0.70147803  0.70288662
  0.70429521]
[0.13261222 0.13262608 0.13263994 ... 0.14644546 0.14645932 0.14647318]
[0.00039237 0.00039387 0.00039536 ... 0.00188602 0.00188752 0.00188901]
[-5.88754018e-05 -5.69286979e-05 -5.49819940e-05 ...  1.88393512e-03
  1.88588182e-03  1.88782852e-03]
[0.27160231 0.27166756 0.27173281 ... 0.3367179  0.33678314 0.33684839]
[0.58369557 0.58373627 0.58377697 ... 0.62431391 0.62435461 0.62439531]
[0.09811461 0.09819421 0.0982738  ... 0.17755154 0.17763114 0.17771074]
[0.1505528  0.15057

[0.02882846 0.02884368 0.02885891 ... 0.04401932 0.04403454 0.04404977]
[0.47548431 0.47549372 0.47550313 ... 0.48487576 0.48488517 0.48489458]
[-0.61858957 -0.61735239 -0.61611522 ...  0.61611522  0.61735239
  0.61858957]
[6.90940216e-05 6.96144522e-05 7.01348829e-05 ... 5.88483785e-04
 5.89004216e-04 5.89524646e-04]
[0.2645629  0.26458546 0.26460802 ... 0.28707746 0.28710002 0.28712258]
[0.00068362 0.00068445 0.00068528 ... 0.00151196 0.00151279 0.00151362]
[0.2661657  0.26618253 0.26619936 ... 0.2829636  0.28298043 0.28299726]
[0.00046802 0.00046856 0.00046911 ... 0.00101071 0.00101125 0.00101179]
[0.00159678 0.00159852 0.00160025 ... 0.00332976 0.0033315  0.00333323]
[0.11267943 0.1129078  0.11313617 ... 0.34059517 0.34082354 0.34105192]
[-6.12390636e-05 -6.08726905e-05 -6.05063173e-05 ...  3.04401321e-04
  3.04767694e-04  3.05134067e-04]
[0.00215678 0.00215855 0.00216032 ... 0.00392498 0.00392675 0.00392852]
[nan nan nan ... nan nan nan]
[0.00147783 0.00147916 0.00148048 ... 0.002

[0.1229927  0.12300563 0.12301856 ... 0.13589677 0.1359097  0.13592263]
[0.00020956 0.00021068 0.00021181 ... 0.00133388 0.00133501 0.00133614]
[nan nan nan ... nan nan nan]
[0.00011009 0.00011083 0.00011157 ... 0.00085032 0.00085106 0.0008518 ]
[0.00061724 0.0006184  0.00061955 ... 0.00176843 0.00176959 0.00177074]
[0.28441841 0.28444146 0.28446451 ... 0.30742166 0.30744471 0.30746776]
[0.21164547 0.21169057 0.21173568 ... 0.25666327 0.25670838 0.25675349]
[0.00018146 0.00018419 0.00018692 ... 0.00290616 0.00290889 0.00291162]
[0.11806583 0.11811856 0.11817129 ... 0.1706926  0.17074533 0.17079807]
[-0.00037464 -0.00037357 -0.0003725  ...  0.00069374  0.00069481
  0.00069588]
[-0.70429521 -0.70288662 -0.70147803 ...  0.70147803  0.70288662
  0.70429521]
[0.21484845 0.21487584 0.21490324 ... 0.24218854 0.24221594 0.24224333]
[0.34061862 0.34064687 0.34067512 ... 0.36881332 0.36884157 0.36886982]
[nan nan nan ... nan nan nan]
[0.20331647 0.20333156 0.20334666 ... 0.21838366 0.21839875 0.

...

[0.20051881 0.2005233  0.2005278  ... 0.20500483 0.20500932 0.20501382]
[0.00014144 0.00014171 0.00014198 ... 0.00041037 0.00041064 0.00041091]
[0.00655394 0.0065577  0.00656146 ... 0.01030565 0.01030941 0.01031316]
[0.00255892 0.00256022 0.00256151 ... 0.00385275 0.00385405 0.00385534]
[0.00037845 0.0003789  0.00037936 ... 0.0008351  0.00083555 0.00083601]
[0.20386935 0.20387494 0.20388054 ... 0.20945496 0.20946056 0.20946615]
[0.00026399 0.00026438 0.00026478 ... 0.00065676 0.00065715 0.00065754]
[0.21302827 0.21303719 0.2130461  ... 0.22192576 0.22193468 0.22194359]
[0.22141911 0.2214339  0.22144869 ... 0.23617864 0.23619343 0.23620822]
[-8.64216045e-06 -7.73959133e-06 -6.83702222e-06 ...  8.92121818e-04
  8.93024387e-04  8.93926957e-04]
[0.00022379 0.00022417 0.00022454 ... 0.00059871 0.00059909 0.00059946]
[0.19292436 0.19294179 0.19295921 ... 0.21031558 0.210333   0.21035043]
[0.16989449 0.1699096  0.16992471 ... 0.18497455 0.18498966 0.18500477]
[0.10967488 0.10968764 0.10970039

[0.2586371  0.25864317 0.25864923 ... 0.26468562 0.26469168 0.26469774]
[0.50553296 0.50553884 0.50554472 ... 0.51140139 0.51140727 0.51141315]
[0.02522014 0.02522406 0.02522799 ... 0.02913266 0.02913658 0.0291405 ]
[0.00026289 0.00026318 0.00026347 ... 0.00055201 0.0005523  0.00055259]
[0.00035907 0.00035946 0.00035985 ... 0.0007464  0.00074678 0.00074717]
[nan nan nan ... nan nan nan]
[0.00033052 0.00033091 0.0003313  ... 0.00072054 0.00072093 0.00072132]
[0.36985865 0.36986644 0.36987422 ... 0.37762431 0.37763209 0.37763987]
[0.0880907  0.08813141 0.08817212 ... 0.12872012 0.12876083 0.12880154]
[0.21727223 0.21728179 0.21729134 ... 0.22680782 0.22681738 0.22682693]
[0.31996862 0.31998348 0.31999833 ... 0.33479232 0.33480717 0.33482203]
[-0.38327759 -0.38251103 -0.38174448 ...  0.38174448  0.38251103
  0.38327759]
[0.06288376 0.06291969 0.06295561 ... 0.09873766 0.09877358 0.09880951]
[-0.2496535  -0.24915419 -0.24865489 ...  0.24865489  0.24915419
  0.2496535 ]
[nan nan nan ... nan

[-0.00047872 -0.00047735 -0.00047598 ...  0.00088649  0.00088786
  0.00088923]
[2.23532550e-05 2.34637355e-05 2.45742160e-05 ... 1.13061278e-03
 1.13172326e-03 1.13283374e-03]
[0.32580305 0.32581955 0.32583605 ... 0.34226587 0.34228237 0.34229887]
[nan nan nan ... nan nan nan]
[0.37261126 0.37263241 0.37265356 ... 0.39372033 0.39374148 0.39376264]
[0.33454512 0.334569   0.33459287 ... 0.35837126 0.35839513 0.35841901]
[0.05845864 0.05862612 0.05879361 ... 0.22561006 0.22577755 0.22594503]
[0.07223827 0.07226    0.07228173 ... 0.09392052 0.09394224 0.09396397]
[-0.00036693 -0.00036322 -0.00035951 ...  0.0033357   0.00333941
  0.00334312]
[-0.38327759 -0.38251103 -0.38174448 ...  0.38174448  0.38251103
  0.38327759]
[0.10665663 0.10681853 0.10698043 ... 0.26823472 0.26839662 0.26855852]
[-0.04424602 -0.04415753 -0.04406904 ...  0.04406904  0.04415753
  0.04424602]
[nan nan nan ... nan nan nan]
[0.00031777 0.00031858 0.0003194  ... 0.00113252 0.00113334 0.00113415]
0.000492473118348379
[0

[0.35303569 0.3530395  0.35304332 ... 0.35684227 0.35684609 0.3568499 ]
[0.00610628 0.00610713 0.00610798 ... 0.00695532 0.00695617 0.00695702]
[0.0014269  0.0014273  0.00142769 ... 0.0018188  0.00181919 0.00181958]
[0.17900568 0.17900863 0.17901158 ... 0.18195097 0.18195392 0.18195688]
[-0.00016836 -0.00016769 -0.00016703 ...  0.00049714  0.00049781
  0.00049848]
[0.35303569 0.3530395  0.35304332 ... 0.35684227 0.35684609 0.3568499 ]
0.3537642017680979
[0.00610628 0.00610713 0.00610798 ... 0.00695532 0.00695617 0.00695702]
0.006272171081942544
[0.0014269  0.0014273  0.00142769 ... 0.0018188  0.00181919 0.00181958]
0.0015050468044540124
[0.17900568 0.17900863 0.17901158 ... 0.18195097 0.18195392 0.18195688]
0.17956935978133454
[-0.00016836 -0.00016769 -0.00016703 ...  0.00049714  0.00049781
  0.00049848]
3.769322900650798e-05
[0.35303569 0.3530395  0.35304332 ... 0.35684227 0.35684609 0.3568499 ]
[0.00610628 0.00610713 0.00610798 ... 0.00695532 0.00695617 0.00695702]
[0.0014269  0.0014

[0.23731489 0.23732026 0.23732564 ... 0.24267576 0.24268113 0.2426865 ]
[0.00875423 0.00875545 0.00875668 ... 0.00997687 0.00997809 0.00997932]
[0.01085262 0.01085605 0.01085949 ... 0.01427959 0.01428303 0.01428646]
[0.31525971 0.31531776 0.3153758  ... 0.37318795 0.37324599 0.37330404]
[0.00082904 0.00083002 0.000831   ... 0.0018067  0.00180768 0.00180866]
[-4.83360203e-05 -4.79869960e-05 -4.76379717e-05 ...  2.99990255e-04
  3.00339280e-04  3.00688304e-04]
[0.00014749 0.00014848 0.00014948 ... 0.0011388  0.00113979 0.00114078]
[0.4647855  0.46480253 0.46481956 ... 0.48178103 0.48179806 0.48181509]
[0.00977522 0.00980433 0.00983344 ... 0.03882665 0.03885576 0.03888487]
[-0.70429521 -0.70288662 -0.70147803 ...  0.70147803  0.70288662
  0.70429521]
[-0.70429521 -0.70288662 -0.70147803 ...  0.70147803  0.70288662
  0.70429521]
[0.34482365 0.34482831 0.34483297 ... 0.34947176 0.34947642 0.34948108]
0.3456247317270164
[0.23731489 0.23732026 0.23732564 ... 0.24267576 0.24268113 0.2426865 ]
